In [ ]:
import pandas as pd
import numpy as np

HIGH_COST = 1e6


class TransportPlanner:
    def __init__(self):
        self.bus_routes = np.array(["BR1", "BR2", "BR3"])
        self.neighborhoods = np.array(["N1", "N2", "N3", "N4"])

        self.neighborhood_demand = np.array([150, 180, 120, 250])
        self.bus_capacity = np.array([200, 300, 250])

        self.costs = np.array(
            [
                [10, 15, 12, 8],
                [10, 12, 8, 11],
                [9, 13, 10, 14],
            ]
        )

        self.total_costs = []

    def find_max_demand_neighborhood(self, allocation):
        return np.argmax(self.neighborhood_demand > np.sum(allocation, axis=0))

    def find_max_capacity_route(self, allocation):
        return np.argmax(self.bus_capacity - np.sum(allocation, axis=1))

    def calculate_available_capacity(self, max_capacity_route, allocation):
        return self.bus_capacity[max_capacity_route] - np.sum(
            allocation[max_capacity_route]
        )

    def can_meet_demand(self, max_demand_neighborhood, available_capacity):
        return self.neighborhood_demand[max_demand_neighborhood] <= available_capacity

    def assign_buses(self, max_capacity_route, max_demand_neighborhood, allocation):
        allocation[max_capacity_route][
            max_demand_neighborhood
        ] += self.neighborhood_demand[max_demand_neighborhood]
        cost = (
            self.neighborhood_demand[max_demand_neighborhood]
            * self.costs[max_capacity_route][max_demand_neighborhood]
        )
        self.total_costs.append(cost)

    def calculate_cost_effectiveness(self):
        cost_effectiveness = np.zeros(self.costs.shape)
        for i in range(len(self.bus_routes)):
            for j in range(len(self.neighborhoods)):
                if self.neighborhood_demand[j] != 0:
                    cost_effectiveness[i, j] = (
                        self.costs[i, j] / self.neighborhood_demand[j]
                    )
                else:
                    cost_effectiveness[i, j] = HIGH_COST

        return cost_effectiveness

    def solve_problem(self):
        allocation = np.zeros(
            (len(self.bus_routes), len(self.neighborhoods)), dtype=int
        )
        cost_effectiveness = self.calculate_cost_effectiveness()
        demand_remaining = self.neighborhood_demand.copy()
        sorted_routes = np.dstack(
            np.unravel_index(
                np.argsort(cost_effectiveness.ravel()), cost_effectiveness.shape
            )
        )[0]

        for route, neighborhood in sorted_routes:
            if demand_remaining[neighborhood] > 0:
                available_capacity = min(
                    self.bus_capacity[route] - np.sum(allocation[route]),
                    demand_remaining[neighborhood],
                )
                allocation[route, neighborhood] += available_capacity
                demand_remaining[neighborhood] -= available_capacity
                cost = available_capacity * self.costs[route, neighborhood]
                self.total_costs.append(cost)

            if np.all(demand_remaining == 0):
                break

        return allocation

    def print_allocation_result(self):
        allocation_result = self.solve_problem()
        df = pd.DataFrame(
            allocation_result, index=self.bus_routes, columns=self.neighborhoods
        )
        print(df)

    def print_total_costs(self):
        print(f"Values found during the solution: {self.total_costs}")
        total_cost = sum(self.total_costs)
        print(f"Total cost: {total_cost}")


if __name__ == "__main__":
    planner = TransportPlanner()
    planner.print_allocation_result()
    planner.print_total_costs()